In [7]:
import librosa
from librosa.core.spectrum import amplitude_to_db
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt

if __name__ == '__main__':
    # 谱减法
    noisy_file_path = './spec_sub_demo.wav'
    noisy, fs = librosa.load(noisy_file_path, sr = None)
    noise = noisy[:64000]
    
    S_noisy = librosa.stft(noisy, n_fft = 256, hop_length = 128, win_length = 256)
    D, T = np.shape(S_noisy)
    Mag_noisy = np.abs(S_noisy)
    Phase_noisy = np.angle(S_noisy)
    Power_noisy = Mag_noisy ** 2
    
    S_noise = librosa.stft(noise, n_fft = 256, hop_length = 128, win_length = 256)
    Mag_noise = np.mean(np.abs(S_noise), axis = 1, keepdims=True)
    Power_noise = Mag_noise ** 2
    Power_noise = np.tile(Power_noise, [1, T])
    
    Power_enhenc = Power_noisy - Power_noise
    Power_enhenc[Power_enhenc<0] = 0
    Mag_enhenc = np.sqrt(Power_enhenc)
    
    S_enhenc = Mag_enhenc * np.exp(1j * Phase_noisy)
    enhenc = librosa.istft(S_enhenc, hop_length=128, win_length=256)
    sf.write('./spec_sub_demo_enhenc.wav', enhenc, fs)

In [12]:
import librosa
from librosa.core.spectrum import amplitude_to_db
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt

if __name__ == '__main__':
    # 过减法
    noisy_file_path = './spec_sub_demo.wav'
    noisy, fs = librosa.load(noisy_file_path, sr = None)
    noise = noisy[:64000]
    
    S_noisy = librosa.stft(noisy, n_fft = 256, hop_length = 128, win_length = 256)
    D, T = np.shape(S_noisy)
    Mag_noisy = np.abs(S_noisy)
    Phase_noisy = np.angle(S_noisy)
    Power_noisy = Mag_noisy ** 2
    
    S_noise = librosa.stft(noise, n_fft = 256, hop_length = 128, win_length = 256)
    Mag_noise = np.mean(np.abs(S_noise), axis = 1, keepdims=True)
    Power_noise = Mag_noise ** 2
    Power_noise = np.tile(Power_noise, [1, T])
    
    alpha, gamma, beta = 4, 1, 0.0001
    
    #Power_enhenc = Power_noisy - Power_noise
    Power_enhenc = np.power(Power_noisy, gamma) - alpha * np.power(Power_noise, gamma)
    Power_enhenc = np.power(Power_enhenc, 1 / gamma)
    mask = (Power_enhenc >= beta * Power_noise) - 0
    Power_enhenc = mask * Power_enhenc + (1 - mask) * beta * Power_noise
    Mag_enhenc = np.sqrt(Power_enhenc)
    
    S_enhenc = Mag_enhenc * np.exp(1j  * Phase_noisy)
    enhenc = librosa.istft(S_enhenc, hop_length=128, win_length=256)
    #sf.write('./spec_sub_demo_enhenc_1.wav', enhenc, fs)

In [13]:
import librosa
from librosa.core.spectrum import amplitude_to_db
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt

if __name__ == '__main__':
    # 过减法-引入平滑
    noisy_file_path = './spec_sub_demo.wav'
    noisy, fs = librosa.load(noisy_file_path, sr = None)
    noise = noisy[:64000]
    
    S_noisy = librosa.stft(noisy, n_fft = 256, hop_length = 128, win_length = 256)
    D, T = np.shape(S_noisy)
    Mag_noisy = np.abs(S_noisy)
    Phase_noisy = np.angle(S_noisy)
    Power_noisy = Mag_noisy ** 2
    
    S_noise = librosa.stft(noise, n_fft = 256, hop_length = 128, win_length = 256)
    Mag_noise = np.mean(np.abs(S_noise), axis = 1, keepdims=True)
    Power_noise = Mag_noise ** 2
    Power_noise = np.tile(Power_noise, [1, T])
    
    Mag_noisy_new = np.copy(Mag_noisy)
    k = 1
    for t in range(k, T-k):
        Mag_noisy_new[:, t] = np.mean(Mag_noisy[:, t-k : t+k+1], axis = 1)
    Power_noisy = Mag_noisy_new ** 2
    
    alpha, gamma, beta = 4, 1, 0.0001
    
    #Power_enhenc = Power_noisy - Power_noise
    Power_enhenc = np.power(Power_noisy, gamma) - alpha * np.power(Power_noise, gamma)
    Power_enhenc = np.power(Power_enhenc, 1 / gamma)
    mask = (Power_enhenc >= beta * Power_noise) - 0
    Power_enhenc = mask * Power_enhenc + (1 - mask) * beta * Power_noise
    Mag_enhenc = np.sqrt(Power_enhenc)
    
    Mag_enhenc_new = np.copy(Mag_enhenc)
    maxnr = np.max(np.abs(S_noise) - Mag_noise, axis = 1)
    for t in range(k, T-k):
        index = np.where(Mag_enhenc[:, t] < maxnr)[0]
        temp = np.min(Mag_enhenc[:, t-k:t+k+1], axis = 1)
        Mag_enhenc_new[index, t] = temp[index]
    
    S_enhenc = Mag_enhenc * np.exp(1j  * Phase_noisy)
    enhenc = librosa.istft(S_enhenc, hop_length=128, win_length=256)
    sf.write('./spec_sub_demo_enhenc_2.wav', enhenc, fs)